<a href="https://colab.research.google.com/github/akandwanaho/contact/blob/master/umojai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Check if GPU is available
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
# Install necessary libraries
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 w

In [3]:
# Import necessary modules
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [4]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [5]:
# Load tokenizer (replace 'bert-base-uncased' with your specific tokenizer)
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Load a larger dataset from Hugging Face Hub
dataset = load_dataset('wikitext', 'wikitext-103-raw-v1')

# Filter out empty lines and short sequences
def filter_function(examples):
    return [len(text.strip()) > 0 and len(text.split()) > 5 for text in examples['text']]

filtered_dataset = dataset['train'].filter(filter_function)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=30)

tokenized_dataset = filtered_dataset.map(tokenize_function, batched=True)

# Shift labels by one position for language modeling
def shift_labels(features):
    features["labels"] = [input_ids[1:] + [tokenizer.pad_token_id] for input_ids in features["input_ids"]]
    return features

final_dataset = tokenized_dataset.map(shift_labels, batched=True)

# Convert dataset to Hugging Face Dataset format
train_dataset = final_dataset
print("✅ Dataset ready with", len(train_dataset), "samples!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Map:   0%|          | 0/1165029 [00:00<?, ? examples/s]

Map:   0%|          | 0/1165029 [00:00<?, ? examples/s]

✅ Dataset ready with 1165029 samples!


In [6]:
import torch
import torch.nn as nn
from transformers import PreTrainedModel, BertConfig

# Define a custom configuration for our model
class UmojaiConfig(BertConfig):
    model_type = "umoja"

# Define the custom model class
class UmojaiModel(PreTrainedModel):
    config_class = UmojaiConfig

    def __init__(self, config):
        super().__init__(config)
        self.transformer = nn.Transformer(
            d_model=config.hidden_size,
            nhead=config.num_attention_heads,
            num_encoder_layers=config.num_hidden_layers,
            num_decoder_layers=config.num_hidden_layers,
            dim_feedforward=4 * config.hidden_size
        )
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size)

    def forward(self, input_ids, attention_mask=None, labels=None):
        # Prepare inputs for the transformer
        src = input_ids.permute(1, 0)  # Transformer expects (seq_len, batch_size)
        tgt = input_ids.permute(1, 0)  # Same for target sequence

        if attention_mask is not None:
            attention_mask = attention_mask.permute(1, 0)  # Adjust mask shape

        # Pass through the transformer
        transformer_output = self.transformer(src, tgt)

        # Reshape output to (batch_size, seq_len, hidden_size)
        transformer_output = transformer_output.permute(1, 0, 2)

        # Apply the language modeling head
        logits = self.lm_head(transformer_output)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            loss = loss_fct(shift_logits.view(-1, config.vocab_size), shift_labels.view(-1))

        return {'loss': loss, 'logits': logits}

# Initialize the model with the tokenizer's vocabulary size
config = UmojaiConfig(vocab_size=len(tokenizer), hidden_size=768, num_hidden_layers=6, num_attention_heads=12)
model = UmojaiModel(config).to(device)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [7]:
import torch
import torch.nn as nn
from transformers import PreTrainedModel, BertConfig

# Define a custom configuration for our model
class UmojaiConfig(BertConfig):
    model_type = "umoja"

# Define the custom model class
class UmojaiModel(PreTrainedModel):
    config_class = UmojaiConfig

    def __init__(self, config):
        super().__init__(config)
        self.transformer = nn.Transformer(
            d_model=config.hidden_size,
            nhead=config.num_attention_heads,
            num_encoder_layers=config.num_hidden_layers,
            num_decoder_layers=config.num_hidden_layers,
            dim_feedforward=4 * config.hidden_size,
            batch_first=True  # Add this line to use batch-first format
        )
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size)

    def forward(self, input_ids, attention_mask=None, labels=None):
        # Prepare inputs for the transformer
        src = input_ids  # No need to permute since batch_first=True
        tgt = input_ids  # Same for target sequence

        if attention_mask is not None:
            attention_mask = attention_mask  # No need to permute since batch_first=True

        # Pass through the transformer
        transformer_output = self.transformer(src, tgt)

        # Reshape output to (batch_size, seq_len, hidden_size) - already in this format
        # Apply the language modeling head
        logits = self.lm_head(transformer_output)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            loss = loss_fct(shift_logits.view(-1, config.vocab_size), shift_labels.view(-1))

        return {'loss': loss, 'logits': logits}

# Initialize the model with the tokenizer's vocabulary size
config = UmojaiConfig(vocab_size=len(tokenizer), hidden_size=768, num_hidden_layers=6, num_attention_heads=12)
model = UmojaiModel(config).to(device)

In [8]:
# Install necessary libraries
!pip install transformers datasets

In [9]:
# Import necessary modules
from datasets import load_dataset
from transformers import AutoTokenizer, PreTrainedModel, BertConfig
import torch
import torch.nn as nn
from transformers import Trainer, TrainingArguments

In [10]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [11]:
# Load tokenizer (replace 'bert-base-uncased' with your specific tokenizer)
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Load a larger dataset from Hugging Face Hub
dataset = load_dataset('wikitext', 'wikitext-103-raw-v1')

# Filter out empty lines and short sequences
def filter_function(examples):
    return [len(text.strip()) > 0 and len(text.split()) > 5 for text in examples['text']]

filtered_dataset = dataset['train'].filter(filter_function)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=30)

tokenized_dataset = filtered_dataset.map(tokenize_function, batched=True)

# Shift labels by one position for language modeling
def shift_labels(features):
    features["labels"] = [input_ids[1:] + [tokenizer.pad_token_id] for input_ids in features["input_ids"]]
    return features

final_dataset = tokenized_dataset.map(shift_labels, batched=True)

# Convert dataset to Hugging Face Dataset format
train_dataset = final_dataset
print("✅ Dataset ready with", len(train_dataset), "samples!")

Map:   0%|          | 0/1165029 [00:00<?, ? examples/s]

✅ Dataset ready with 1165029 samples!


In [12]:
# Define a custom configuration for our model
class UmojaiConfig(BertConfig):
    model_type = "umoja"

# Define the custom model class
class UmojaiModel(PreTrainedModel):
    config_class = UmojaiConfig

    def __init__(self, config):
        super().__init__(config)
        self.transformer = nn.Transformer(
            d_model=config.hidden_size,
            nhead=config.num_attention_heads,
            num_encoder_layers=config.num_hidden_layers,
            num_decoder_layers=config.num_hidden_layers,
            dim_feedforward=4 * config.hidden_size,
            batch_first=True  # Add this line to use batch-first format
        )
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size)

    def forward(self, input_ids, attention_mask=None, labels=None):
        # Prepare inputs for the transformer
        src = input_ids  # No need to permute since batch_first=True
        tgt = input_ids  # Same for target sequence

        if attention_mask is not None:
            attention_mask = attention_mask  # No need to permute since batch_first=True

        # Pass through the transformer
        transformer_output = self.transformer(src, tgt)

        # Reshape output to (batch_size, seq_len, hidden_size) - already in this format
        # Apply the language modeling head
        logits = self.lm_head(transformer_output)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            loss = loss_fct(shift_logits.view(-1, config.vocab_size), shift_labels.view(-1))

        return {'loss': loss, 'logits': logits}

# Initialize the model with the tokenizer's vocabulary size
config = UmojaiConfig(vocab_size=len(tokenizer), hidden_size=768, num_hidden_layers=6, num_attention_heads=12)
model = UmojaiModel(config).to(device)

In [13]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,  # Replace with your validation dataset if available
)

# Start training
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: stephen-akandwanaho (stephen-akandwanaho-richfield) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


RuntimeError: the feature number of src and tgt must be equal to d_model

In [14]:
import torch
import torch.nn as nn
from transformers import PreTrainedModel, BertConfig

# Define a custom configuration for our model
class UmojaiConfig(BertConfig):
    model_type = "umoja"

# Define the custom model class
class UmojaiModel(PreTrainedModel):
    config_class = UmojaiConfig

    def __init__(self, config):
        super().__init__(config)
        self.embedding = nn.Embedding(config.vocab_size, config.hidden_size)  # Add embedding layer
        self.transformer = nn.Transformer(
            d_model=config.hidden_size,
            nhead=config.num_attention_heads,
            num_encoder_layers=config.num_hidden_layers,
            num_decoder_layers=config.num_hidden_layers,
            dim_feedforward=4 * config.hidden_size,
            batch_first=True  # Use batch-first format
        )
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size)

    def forward(self, input_ids, attention_mask=None, labels=None):
        # Embed the input ids
        src = self.embedding(input_ids)  # Embed the input tokens
        tgt = self.embedding(input_ids)  # Same for target sequence

        if attention_mask is not None:
            attention_mask = attention_mask  # No need to permute since batch_first=True

        # Pass through the transformer
        transformer_output = self.transformer(src, tgt)

        # Apply the language modeling head
        logits = self.lm_head(transformer_output)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            loss = loss_fct(shift_logits.view(-1, config.vocab_size), shift_labels.view(-1))

        return {'loss': loss, 'logits': logits}

# Initialize the model with the tokenizer's vocabulary size
config = UmojaiConfig(vocab_size=len(tokenizer), hidden_size=768, num_hidden_layers=6, num_attention_heads=12)
model = UmojaiModel(config).to(device)

In [15]:
# Load tokenizer (replace 'bert-base-uncased' with your specific tokenizer)
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Load a larger dataset from Hugging Face Hub
dataset = load_dataset('wikitext', 'wikitext-103-raw-v1')

# Filter out empty lines and short sequences
def filter_function(examples):
    return [len(text.strip()) > 0 and len(text.split()) > 5 for text in examples['text']]

filtered_dataset = dataset['train'].filter(filter_function)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=30)

tokenized_dataset = filtered_dataset.map(tokenize_function, batched=True)

# Shift labels by one position for language modeling
def shift_labels(features):
    features["labels"] = [input_ids[1:] + [tokenizer.pad_token_id] for input_ids in features["input_ids"]]
    return features

final_dataset = tokenized_dataset.map(shift_labels, batched=True)

# Convert dataset to Hugging Face Dataset format
train_dataset = final_dataset
print("✅ Dataset ready with", len(train_dataset), "samples!")

✅ Dataset ready with 1165029 samples!


In [ ]:
from transformers import Trainer, TrainingArguments

# Define training arguments with run_name and disabling WandB
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Changed from evaluation_strategy to eval_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    run_name="umoja_run",
    report_to=[],  # Disable WandB by clearing the report_to list
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,  # Replace with your validation dataset if available
)

# Start training
trainer.train()

Epoch,Training Loss,Validation Loss
